In [2]:
import importlib
from tools import network_tools as network_tools
from tools import firewall_tools as firewall_tools
importlib.reload(network_tools)
importlib.reload(firewall_tools)

<module 'tools.firewall_tools' from '/home/federico/Desktop/Thesis/AI_Agent_for_honeypot_operations/HoneypotAgentApp/MultiAgentArchitecture/tools/firewall_tools.py'>

In [17]:
alerts = await network_tools.get_alerts(time_window=5)
len(alerts.get('security_events', []).get('alerts'))

30

In [23]:
alerts = await network_tools.get_alerts(time_window=2)
alerts

{'security_events': {'alerts': [],
  'time_window_minutes': 2,
  'timestamp': '2025-09-13T23:21:56.127059Z',
  'total_events': 0}}

In [5]:
network_tools.get_docker_containers()

{'honeypot_config': [{'id': 'e688667fd2bc',
   'name': 'unauthorized-rce-docker-1',
   'image': 'unauthorized-rce-docker:latest',
   'status': 'running',
   'created': '2025-09-13T21:09:11.951194833Z',
   'ports': ['2375/tcp', '2376/tcp'],
   'ip_address': '172.20.0.2'},
  {'id': '7014be278f27',
   'name': 'xdebug-rce-xdebug2-1',
   'image': 'vulhub/php:7.1-xdebug',
   'status': 'running',
   'created': '2025-09-13T21:09:10.497200612Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.7'},
  {'id': '0d54748ba2bb',
   'name': 'cve-2021-22205-gitlab-1-proxy',
   'image': 'nginx:alpine',
   'status': 'running',
   'created': '2025-09-13T14:23:30.951434695Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.10'},
  {'id': 'dd34d2021a49',
   'name': 's2-057-struts2-1',
   'image': 'vulhub/struts2:2.3.34-showcase',
   'status': 'running',
   'created': '2025-09-13T14:23:30.254001229Z',
   'ports': ['8080/tcp'],
   'ip_address': '172.20.0.3'}]}

In [22]:
print(network_tools.check_services_health())
rules = await firewall_tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

{'firewall_status': 'up', 'suricata_status': 'up'}
Chain FORWARD (policy DROP)
num  target     prot opt source               destination         
1    ACCEPT     all  --  0.0.0.0/0            0.0.0.0/0            state RELATED,ESTABLISHED
2    ACCEPT     icmp --  0.0.0.0/0            0.0.0.0/0           
3    ACCEPT     all  --  172.20.0.0/24        172.20.0.0/24       
4    DROP       all  --  192.168.100.0/24     172.20.0.0/24       
5    DROP       all  --  172.20.0.0/24        192.168.100.0/24    
6    LOG        all  --  0.0.0.0/0            0.0.0.0/0            LOG flags 0 level 4 prefix "FIREWALL-DROP: "



In [21]:
await firewall_tools.remove_firewall_rule([1, 2])

INFO:tools.firewall_tools:Removing firewall rules: [1, 2]


{'rules_removed_current_epoch': ['REMOVED rule #1:    ACCEPT     tcp  --  172.20.0.2           192.168.100.0/24',
  'REMOVED rule #2:    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.2']}

In [17]:
honeypot_ip = '172.20.0.7' 
attacker_ip = '192.168.100.2/24'

await firewall_tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip, port=8080)
await firewall_tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools.firewall_tools:Adding allow rule: ALLOW 192.168.100.2/24 -> 172.20.0.7 (tcp)
INFO:tools.firewall_tools:Adding allow rule: ALLOW 172.20.0.7 -> 192.168.100.2/24 (tcp)


{'rules_added_current_epoch': 'ALLOW 172.20.0.7 -> 192.168.100.2/24 (tcp)'}